In [1]:
# the server code from micktwomey
import multiprocessing
import socket

In [2]:
def handle(connection, address):
    import logging
    logging.basicConfig(level=logging.DEBUG)
    logger = logging.getLogger("process-%r" % (address, ))
    # like the logger
    try:
        logger.debug("Connected %r at %r", connection, address)
        while True:
            data = connection.recv(1024)
            if data == "":
                logger.debug("Socket closed remotely")
                break
            logger.debug("Received data %r", data)
            connection.sendall(data)
            logger.debug("Send data")
    except:
        logger.exception("Problem handling request")
    finally:
        logger.debug("Closing socket")
        connection.close()


In [3]:
class Server(object): 
    def __init__(self, hostname, port):
        import logging
        self.logger = logging.getLogger("Server")
        self.hostname = hostname
        self.port = port
    def start(self):
        self.logger.debug("Listening")
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket = bind((self.hostname, self.port))
        self.socket.listen(1)
        while True:
            conn, address = self.socket.accept()
            self.logger.debug("Got connection")
            process = multiprocessing.Process(target=handle, args=(conn, address))
            process.daemon = True
            process.start()
            self.logger.debug("Started process %r", process)

In [4]:
if __name__ == "__main__":
    import logging
    logging.basicConfig(level=logging.DEBUG)
    server = Server("0.0.0.0", 9000)
    try: 
        logging.info("Listening")
        server.start()
    except:
        logging.exception("Unexpected exception")
    finally:
        logging.info("Shutting down")
        for process in multiprocessing.active_children():
            logging.info("Shutting down process %r", process)
            process.terminate()
            process.join()
    logging.info("All done")

In [5]:
import logging
logging.basicConfig(level=logging.DEBUG)
server = Server("0.0.0.0", 9000)
try: 
    logging.info("Listening")
    server.start()
except:
    logging.exception("Unexpected exception")
finally:
    logging.info("Shutting down")
    for process in multiprocessing.active_children():
        logging.info("Shutting down process %r", process)
        process.terminate()
        process.join()
logging.info("All done")